In [2]:
import pandas as pd
import numpy as np

# Creating the dataframe to collect all the data.
blank_cols = np.arange(1,12,1)
fluor_conc = []
for i in range(0,11):
    fluor_conc.append(10/(2**i))
row_data = {
    'Column' : blank_cols,
    'Fluor_conc' : fluor_conc
}
blank_corrected = pd.DataFrame(row_data)





In [3]:
dfs = {}
for i in range(1,16):
    dfs[f"df{i}"] = pd.read_excel(
        io = '/Users/wilsonporteus/Documents/GitHub/ssb_s5/Main Assesment/Data/bb'+str(i)+'.xlsx',
        header=0,
        sheet_name = 'End point', usecols="B:L",
        skiprows=13,
        nrows=9, engine="openpyxl"
        )
    avg = []
    for col in range(0,11):
        avg.append(
            np.average(dfs[f"df{i}"].iloc[:,col])
        )
    blank_corrected.insert(loc=i+1, column=f"df{i}_bc", value=avg)

In [ ]:
log_df = np.log10(blank_corrected)


      Column  Fluor_conc    df1_bc    df2_bc    df3_bc    df4_bc    df5_bc  \
0   0.000000     1.00000  4.911089  4.917130  4.917517  4.914612  4.899380   
1   0.301030     0.69897  4.608879  4.632012  4.611127  4.624970  4.608275   
2   0.477121     0.39794  4.340514  4.266346  4.302420  4.357165  4.363320   
3   0.602060     0.09691  4.070703  3.979901  4.020465  4.043839  4.081230   
4   0.698970    -0.20412  3.801464  3.693089  3.818407  3.825759  3.797951   
5   0.778151    -0.50515  3.536921  3.455568  3.534502  3.565951  3.518251   
6   0.845098    -0.80618  3.284628  3.293998  3.338929  3.242635  3.254910   
7   0.903090    -1.10721  3.032216  3.160993  3.160956  3.003245  2.977266   
8   0.954243    -1.40824  2.779416  3.024434  2.910624  2.738483  2.707464   
9   1.000000    -1.70927  2.528756  2.957368  2.744097  2.459769  2.431766   
10  1.041393    -2.01030  2.278754  2.824533  2.543136  2.206151  2.156473   

      df6_bc    df7_bc    df8_bc    df9_bc   df10_bc   df11_bc 

In [6]:
slopes = []
r2_vals = []
x = log_df["Fluor_conc"]

for col in range(2,17):
    y = log_df.iloc[:,col]
    m, b = np.polyfit(x, y, 1)
    y_pred = m * x + b
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - (ss_res / ss_tot)
    slopes.append(m)
    r2_vals.append(r2)

exp_number = np.arange(1,16,1)
final_data = {
    "Expirement Number" : exp_number,
    "Gradient" : slopes,
    "R2" : r2_vals
}
final_df = pd.DataFrame(final_data)

final_df.to_excel("BB_gradient.xlsx")

In [7]:
stdev = pd.DataFrame(row_data)

dfs1 = {}
for i in range(1,16):
    dfs1[f"df{i}"] = pd.read_excel(
        io = '/Users/wilsonporteus/Documents/GitHub/ssb_s5/Main Assesment/Data/bb'+str(i)+'.xlsx',
        header=0,
        sheet_name = 'End point', usecols="B:L",
        skiprows=13,
        nrows=9, engine="openpyxl"
        )
    s = []
    for col in range(0,11):
        s.append(
            np.std(dfs[f"df{i}"].iloc[:,col])
        )
    stdev.insert(loc=i+1, column=f"df{i}_bc", value=s)



w = stdev / blank_corrected

#print(w)

list = []
for x in range (2,17):
    a = np.average(w.iloc[:,x])
    list.append(a)

print(list)

[np.float64(0.10479670632084594), np.float64(0.15128895347318075), np.float64(0.14369423164725384), np.float64(0.1723751558502456), np.float64(0.06563473260536404), np.float64(0.10305441124999942), np.float64(0.3564943876975794), np.float64(0.033891107462717424), np.float64(0.1459028099673797), np.float64(0.019413318517806408), np.float64(0.09456680284698232), np.float64(0.19456458469468885), np.float64(0.09275442742839025), np.float64(0.0647119442093096), np.float64(0.08282234827400235)]
